# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Workspace, Experiment
# ws = Workspace.from_config()
### YOUR CODE HERE ###

# compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
# compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

In [2]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
cpu_cluster_name = "hyperdrive-compute"
compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
experiment_name = 'hyperdrive-capstone'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

#TODO: Create the different params that you will be using during training
param_sampling =  RandomParameterSampling( {
        "--C": uniform(0,2),
        "--max_iter": choice(25,50,75,100,125,150,175,200)
    })### YOUR CODE HERE ###
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            arguments=['--C', 1, '--max_iter', 100],
                            compute_target=compute_target,
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator, 
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=10,
                             max_concurrent_runs=4)

In [12]:
#TODO: Submit your experiment
hyper_drive= experiment.submit(config=hyperdrive_run_config)
RunDetails(hyper_drive).show()
hyper_drive.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce
Web View: https://ml.azure.com/runs/HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-252189/workspaces/quick-starts-ws-252189&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-02-09T07:39:44.724967][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-02-09T07:39:45.1438813Z][SCHEDULER][INFO]Scheduling job, id='HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_0' 
[2024-02-09T07:39:45.3289657Z][SCHEDULER][INFO]Scheduling job, id='HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_2' 
[2024-02-09T07:39:45.2928221Z][SCHEDULER][INFO]Scheduling job, id='HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_1' 
[2024-02-09T07:39:45.4647396Z][SCHEDULER][INFO]Scheduling job, id='HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_3' 
[2024-02-09T07:39:45.417124][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce',
 'target': 'hyperdrive-compute',
 'status': 'Completed',
 'startTimeUtc': '2024-02-09T07:39:43.971252Z',
 'endTimeUtc': '2024-02-09T07:42:46.39096Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e2257d23-a390-4d6f-9699-2cec07144048',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_2',
  'score': '0.7676767676767676',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemet

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
best_run = hyper_drive.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[5])
print('\n max_iter:',parameter_values[7])

Best Run Id:  HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_2

 Accuracy: 0.7676767676767676

 C: 0.036230501710964536

 max_iter: 200


In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-capstone,HD_13d01ebc-f327-448b-8927-a9c6ac56d2ce_2,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
#TODO: Save the best model
best_auto_run_metrics = best_run.get_metrics()
auto_model = best_run.register_model(model_name='hyper_drive_model', model_path='/', 
                                properties={'Accuracy': best_run_metrics['Accuracy']})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

